In [2]:
import BioSimSpace as BSS
import glob
import csv
import numpy as np

/home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)



Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



In [4]:
number_of_ligands = 16
ligand_path = "kpc2/ligands/"
protein_path = "kpc2/protein/"

# Try LOMAP

In [5]:
ligand_files = glob.glob(ligand_path+"*.mol2")

In [6]:
ligand_files

['kpc2/ligands/lig_h_8.mol2',
 'kpc2/ligands/lig_h_4.mol2',
 'kpc2/ligands/lig_h_13.mol2',
 'kpc2/ligands/lig_h_16.mol2',
 'kpc2/ligands/lig_h_9.mol2',
 'kpc2/ligands/lig_h_6.mol2',
 'kpc2/ligands/lig_h_15.mol2',
 'kpc2/ligands/lig_h_10.mol2',
 'kpc2/ligands/lig_h_7.mol2',
 'kpc2/ligands/lig_h_12.mol2',
 'kpc2/ligands/lig_h_3.mol2',
 'kpc2/ligands/lig_h_1.mol2',
 'kpc2/ligands/lig_h_5.mol2',
 'kpc2/ligands/lig_h_2.mol2',
 'kpc2/ligands/lig_h_14.mol2',
 'kpc2/ligands/lig_h_11.mol2']

In [34]:
ligands = []
ligand_names = []

for file in ligand_files:
    ligand = BSS.IO.readMolecules(file)[0]
    # print(ligand)
    ligands.append(ligand)
    ligand_names.append(file.split("/")[-1].replace(".mol2", ""))
ligand_names
# transformations, lomap_scores = BSS.Align.generateNetwork(ligands, plot_network=True, names=ligand_names)

['lig_h_8',
 'lig_h_4',
 'lig_h_13',
 'lig_h_16',
 'lig_h_9',
 'lig_h_6',
 'lig_h_15',
 'lig_h_10',
 'lig_h_7',
 'lig_h_12',
 'lig_h_3',
 'lig_h_1',
 'lig_h_5',
 'lig_h_2',
 'lig_h_14',
 'lig_h_11']

Doesn't work!!!

# `ligands.dat` file

Create a folder `./execution_model/` and then:

In [13]:
with open("./execution_model/ligands.dat", "w") as ligands_file:
    writer = csv.writer(ligands_file)
    for ligand in ligand_names:
        writer.writerow([ligand])

# `network.dat` file 

In [23]:
np.set_printoptions(formatter={'float': '{: .4f}'.format})

In [24]:
engine = "SOMD"

In [26]:
lambda_array = np.around(np.linspace(0, 1, 11), decimals=5)
n_windows = len(lambda_array)

In [37]:
centre = ""
perturbations = []
for name in ligand_names:
    if name == "lig_h_1":
        centre = name
    else:
        perturbations.append(name)
# print(centre)
# print("---")
# print(perturbations)

In [39]:
with open("./execution_model/network.dat", "w") as network_file:
    
    writer = csv.writer(network_file, delimiter=" ")

    for perturbation in perturbations:

        bash_lambda_array = str(lambda_array).replace("[ ", "").replace("]", "").replace("  ", ",").replace('\n', '')
        bash_lambda_array
        # # write out both directions for this perturbation.
        writer.writerow([centre, perturbation, n_windows, bash_lambda_array, engine])
        
        # Uncomment for reverse perturbation!
        # writer.writerow([perturbation, centre, n_windows, bash_lambda_array, engine])      

# `protocol.dat` file

In [42]:
protocol = [
    f"ligand forcefield = gaff2",
    f"protein forcefield = ff14SB",
    f"solvent = TIP3P",
    f"box edges = 20*angstrom",
    f"box type = orthorhombic",
    f"protocol = default",
    f"sampling = 2*ns",
    f"engine = SOMD"
]

with open("./execution_model/protocol.dat", "w") as protocol_file:
    writer = csv.writer(protocol_file)

    for line in protocol:
        writer.writerow([line])

# Protein parameterisation

In [45]:
protein_dry = BSS.IO.readPDB(protein_path+"kpc2.amber.pdb")[0]
protein_dry = BSS.Parameters.parameterise(protein_dry, "ff14SB").getMolecule()
crystal_water = BSS.IO.readMolecules(protein_path+"hoh.pdb")[0]
protein = protein_dry + crystal_water

<BioSimSpace.Molecules: nMolecules=2>

# Save molecules

In [47]:
BSS.IO.saveMolecules("execution_model/inputs/protein/protein", protein, fileformat="pdb")

['/home/jguven/Projects/metalloproteins/execution_model/inputs/protein/protein.pdb']

# Parameterise protein + water system on `tleap`

See notion for details.

Saved 

In [10]:
protein = BSS.IO.readMolecules(["execution_model/inputs/protein/protein_complex.prm7", "execution_model/inputs/protein/protein_complex.rst7"])[0]

In [12]:
# protein.isPerturbable()